本文介绍传统机器学习算法中的“支撑向量机（SVM）”，在深度学习变得流行之前，SVM是许多论文中经常会用到的算法。它有非常完备的数学推导，我在刚接触它的时候也被搞得云里雾里。现在打算系统的介绍一下它。

本文一共分成以下几个部分
1. 线性可分支撑向量机及其对偶问题
2. 线性不可分支撑向量机及其对偶问题
3. 非线性支撑向量机（核技巧）
4. SMO算法

文章重点参考了《统计学习方法》的支撑向量机一章。

**声明**：文章中用到的上下标，上标的标示是括号加数字，例如$\textbf{x}^{(i)}$表示样本集中第i个样本点，加粗的$\textbf{x}$表示这是一个向量。下标$x_i$表示向量$\textbf{x}$的第i个维度。这种标记方式是follow ng的课程的。

## 1. 线性可分支撑向量机及其对偶问题
这是一个最简单的SVM算法，通过这个例子，将会认识到利用SVM算法求解一个简单的二分类问题的整个流程。后面的线性不可分问题以及核技巧，都是在基于线性可分的基础上进行的拓展。所以这个环节会介绍的比较详细，我会尽量讲清楚公式推导的部分，不要畏惧公式的繁杂，慢慢看是可以看明白的。

### 1.1 问题描述
给定一组训练数据集

\begin{equation}
T=\{(\textbf{x}^{(1)},y^{(1)}),(\textbf{x}^{(2)},y^{(2)}),\cdots, (\textbf{x}^{(N)}, y^{(N)})\}
\end{equation}

该数据集包含N个样本点。每个样本点的$\textbf{x}$加粗显示，表示这是一个向量，$\textbf{x}\in \mathbb{R}^{n}$，当然如果n=1，则$\textbf{x}$是一个标量。

在$\textbf{x}^{(i)}=(x^{(i)}_1,x^{(i)}_2,\cdots, x^{(i)}_n)$中的每一个维度，表示该样本点的一个特征，样本集中的每个样本点有n个维度或特征。

$y^{(i)}$表示第i个样本点的类别，$y\in\{+1, -1\}$，当$y^{(i)}=1$，则表示$x^{(i)}$是正例。当$y^{(i)}=-1$，则表示$x^{(i)}$是负例。学习的目标就是要找到一个超平面，这个超平面把空间分成两个部分，使得样本集中的正负样本点分别位于各自的部分。这个超平面用方程表示为$\textbf{w}\cdot \textbf{x}+b=0$，它由法相量$\textbf{w}$和截距项$b$所决定。

需要注意的是，对于一组线性可分的数据集，存在着无穷多个超平面可以把这组数据集的正负例完全分开。我们要找的是一个最优的超平面，以下将解释什么样的超平面算最优。

<img src="img/二分类问题样例.png">

### 1.2 最优超平面
上面介绍到，超平面由法相量$\textbf{w}$和截距项$b$所决定，那么这里定义最优超平面对应的法相量和截距项分别为$\textbf{w}^{\star}$和$b^{\star}$。则最优超平面的方程为
$$
\begin{equation}
\textbf{w}^{\star}\cdot \textbf{x} + b^{\star}=0
\label{classifier}
\end{equation}
$$
在上面的图7.1中，样本点的特征是二维的，所以利用一条直线可以将样本点分割开。用o表示正例，x表示负例。在正例中有三个点A、B、C，可以看到这三个点都被直线正确的分类了。但是仔细看这三个点到直线的距离则有所不同，C离直线的距离最近，A离得最远，B居中，这里说的距离可以理解为垂直于直线的距离。当距离越大时，表示分类的结果比较可信，当距离越小时，表示分类的结果比较不可信。为什么要说可信不可信，是因为我们获得的样本集T只是从众多样本点中随机抽样得到的，假如在运气不好的情况下，获得的样本集比较诡异，那么虽然分类直线在这个样本集上做到了完全分类正确，但是这条分类直线在其他不在样本集上的样本点，可能表现就会不佳。因为我们在抽样的时候没有做好，使得样本无法代表总体。而基于这样的样本集得到的分类直线，则不是一条好的分类直线。


在这个例子中，C点距离直线最近，表示虽然直线把C点的类别分对了，但是差一点就分错了。因为有上述抽样问题存在，我们不敢把C点刚刚好分对，而要尽可能的让它远离分类直线。通过使得这个距离最大化，得到最优的分类超平面。

为了解释最优超平面，首先要引入函数间隔和几何间隔的概念。

#### 函数间隔
超平面的方程式是$\textbf{w}\cdot \textbf{x} + b=0$，则样本点$(\textbf{x}^{(i)}, y^{(i)})$到超平面$(\textbf{w}, b)$的函数间隔为
\begin{equation}
\hat{\gamma}^{(i)}= y^{(i)}(\textbf{w}\cdot \textbf{x}^{(i)} + b)
\end{equation}
为什么可以这么定义，原因是对于超平面$(\textbf{w}, b)$而言，样本点$(\textbf{x}^{(i)}, y^{(i)})$到超平面的距离用$\textbf{w}\cdot \textbf{x}+b$表示，这是一个描述距离的值，它的绝对值表示距离平面的远近。这个值可正可负。如果样本点在平面的上方，即平面把它判成正例，则该值是正的，如果样本点在平面的下方，即平面把它判成负例，则该值是负的。

通过比较$\textbf{w}\cdot \textbf{x}+b$的符号与$y^{(i)}$的符号是否相同，可以判断平面是否正确分类。所以我们用$y^{(i)}(\textbf{w}\cdot \textbf{x}^{(i)} + b)$来表示分类正确和距离平面远近，也就是上文函数间隔的定义。

在样本集T中的所有样本点中找到$\hat{\gamma}^{(i)}$最小的一个，即为超平面$(\textbf{w}, b)$关于样本集T的函数间隔
\begin{equation}
\hat{\gamma} = \min_{i=1\cdots N}{\hat{\gamma}^{(i)}}
\end{equation}

#### 几何间隔
上文中的分离超平面方程式$(\textbf{w}, b)：\textbf{w} \cdot \textbf{x} + b=0$，如果在方程两边同时乘以一个系数c，则不改变分离超平面。但是函数间隔却会变成原来的c倍。为了保证样本点到超平面的距离不受到系数的影响，引入几何间隔的概念，即在计算距离的时候对超平面方程式中的参数做归一化。

$$
\begin{equation}
\begin{aligned}
\gamma^{(i)} &= y^{(i)}\frac{(\textbf{w}\cdot \textbf{x}^{(i)} + b)}{\|\textbf{w}\|} \\
 &=y^{(i)}\left(\frac{\textbf{w}}{\|\textbf{w}\|} \cdot \textbf{x}^{(i)} + \frac{b}{\|\textbf{w}\|}\right)
\end{aligned}
\end{equation}
$$

则分离超平面关于样本集T的几何间隔为
$$
\begin{equation}
\gamma = \min_{i=1\cdots N}{\gamma}^{(i)}
\end{equation}
$$

所以函数间隔和集合间隔的关系是

$$
\begin{equation}
\begin{aligned}
\gamma = \frac{\hat{\gamma}}{\|\textbf{w}\|} \\
\gamma^{(i)} = \frac{\hat{\gamma}^{(i)}}{\|\textbf{w}\|}
\end{aligned}
\end{equation}
$$

#### 间隔最大化
这里的间隔指几何间隔最大化。对于一个线性可分的数据集T，存在无穷多个完全分类正确的超平面，通过使得几何间隔$\gamma$最大化，找到最优的分离超平面$(\textbf{w}^{\star}, b^{\star})$

我们可以把上述过程用数学方式表达出来

$$
\begin{equation}
\begin{aligned}
\max_{\textbf{w}, b} \quad &\gamma \\
\text{s. t. } \quad &y^{(i)}\left(\frac{\textbf{w}}{\|\textbf{w}\|} \cdot \textbf{x}^{(i)} + \frac{b}{\|\textbf{w}\|}\right) \geq \gamma, \quad i=1,2,\cdots, N
\end{aligned}
\end{equation}
$$

根据函数间隔与几何间隔之间的关系，我们也可以把上式写成
$$
\begin{equation}
\begin{aligned}
\max_{\textbf{w}, b} \quad &\frac{\hat{\gamma}}{\|\textbf{w}\|} \\
\text{s. t. } \quad &y^{(i)}\left(\textbf{w} \cdot \textbf{x}^{(i)} + b\right) \geq \hat{\gamma}, \quad i=1,2,\cdots, N
\end{aligned}
\end{equation}
$$

上文提到如果用到函数间隔，要注意超平面的系数$\textbf{w}$和$b$按照系数c等比例增大或缩小的问题。假设将$\textbf{w}$和$b$等比例的变成$c\textbf{w}$和$cb$，则函数间隔变成了$c\hat{\gamma}$。带入用函数间隔表示的目标问题和约束条件中后发现没有产生任何影响。所以不失一般性，这里取函数间隔$\hat{\gamma}=1$

所以问题变成
$$
\begin{equation}
\begin{aligned}
\max_{\textbf{w}, b} \quad &\frac{1}{\|\textbf{w}\|} \\
\text{s. t. } \quad &y^{(i)}\left(\textbf{w} \cdot \textbf{x}^{(i)} + b\right) -1 \geq 0, \quad i=1,2,\cdots, N
\end{aligned}
\end{equation}
$$

为了后续推导的方便，利用最小化$\frac{1}{\|\textbf{w}\|}$和最大化$\frac{1}{2}\|\textbf{w}\|^2$是等价的，于是把问题修改成
$$
\begin{equation}
\begin{aligned}
\min_{\textbf{w}, b} \quad &\frac{1}{2}\|\textbf{w}\|^2 \\
\text{s. t. } \quad &y^{(i)}\left(\textbf{w} \cdot \textbf{x}^{(i)} + b\right) -1 \geq 0, \quad i=1,2,\cdots, N
\label{primal}
\end{aligned}
\end{equation}
$$

这是一个目标函数为二次函数，约束条件为一次的凸二次规划问题。通过求解该问题得到$(\textbf{w}^{\star}, b^{\star})$，则最优超平面为$\textbf{w}^{\star}\cdot \textbf{x} + b^{\star}=0$

### 1.3 拉格朗日对偶性质
在正式求解上述凸二次规划问题之前，要先补充一下有关怎么利用对偶问题来求解原始问题的知识。

以下内容都以输入一个样本点$\textbf{x}$为例子。

#### 拉格朗日极小极大问题

考虑以下优化问题
$$
\begin{equation}
\begin{aligned}
\max_{\textbf{x}\in \mathbb{R}^n}\quad &f(\textbf{x}) \\
\text{s.t } \quad &c^{(i)}(\textbf{x}) \leq 0 \quad i=1,2,\cdots, k \\
&h^{(j)}(\textbf{x}) = 0 \quad j=1,2,\cdots, l
\end{aligned}
\end{equation}
$$
这是一个标准的带约束的优化问题，称为原始问题(primal problem)。下面我们要利用拉格朗日对偶性质求解该问题。


假设$f(\textbf{x})$，$c(\textbf{x})$和$h(\textbf{x})$在$\mathbb{R}^n$上是连续可微的（函数存在导函数，且导函数是连续的）。引入拉格朗日函数

$$
\begin{equation}
L(\textbf{x},\alpha,\beta) = f(\textbf{x}) + \sum_{i=1}^k{\alpha^{(i)}c^{(i)}(\textbf{x})} + \sum_{j=1}^l{\beta^{(j)}h^{(j)}(\textbf{x})}
\end{equation}
$$

其中$\alpha^{(i)}$和$\beta^{(j)}$是拉格朗日乘子，且$\alpha^{(i)} \geq 0$。
拉格朗日函数$L(\textbf{x},\alpha,\beta)$有三个变量，分别是$\textbf{x}$，$\alpha$和$\beta$

定义关于$\textbf{x}$的函数
$$
\begin{equation}
\theta_{P}(\textbf{x}) = \max_{\alpha,\beta;\alpha^{(i)}\geq 0}L(\textbf{x},\alpha, \beta)
\end{equation}
$$
下表P表示primal原始问题的意思。可以看到，这个$\theta_P(\textbf{x})$函数从拉格朗日函数演变而来。它只有一个变量$\textbf{x}$，因为剩余的两个变量$\alpha$和$\beta$在求$\max_{\alpha,\beta;\alpha^{(i)}\geq 0}L(\textbf{x},\alpha, \beta)$过程中找到了能够使拉格朗日函数最大的$\alpha$和$\beta$。这里用$\alpha^{\star}$和$\beta^{\star}$表示。所以$\alpha$和$\beta$变量已经固定，即为常数。

分析$\theta_P(\textbf{x})$的取值范围。

**如果存在$\textbf{x}$使得在原始问题中的约束条件不满足**，即存在$c^{(i)}(\textbf{x}) > 0$或$h^{(i)}(\textbf{x}) \neq 0$，则可以通过调节$\alpha$和$\beta$使得$\theta_P(\textbf{x})$函数达到无穷大。例如$c^{(i)}(\textbf{x}) > 0$，则令对应的$\alpha^{(i)}\rightarrow +\infty$。如果$h^{(j)}(\textbf{x}) \neq 0$，则根据符号的方向决定$\beta^{(j)}$的方向，使得$\beta^{(j)}h^{(j)}(\textbf{x}) \rightarrow +\infty$，将其他拉格朗日乘子取0即可。

**如果所有的$\textbf{x}$都满足在原始问题中的约束条件**，则将所有的$\alpha^{(i)}$取0，可以使得$\theta_P(\textbf{x})$取到最大值，即$\theta_P(\textbf{x})=f(\textbf{x})$。

综合考虑两种情况得到$\theta_P{\textbf{x}}$的取值范围
$$
\begin{equation}
\theta_P(\textbf{x})=
 \begin{cases}
   f(\textbf{x}), &\textbf{x}\text{ satisfy the constraints}\\
   +\infty, &\text{otherwise}
   \end{cases}
\end{equation}
$$

可以看到如果$\textbf{x}$满足原始问题的约束条件，则$\theta_P(\textbf{x})$可以取到$f(\textbf{x})$。原始问题是通过调节$\textbf{x}$对$f(\textbf{x})$进行最小化，则通过调节$\textbf{x}$来最小化$\theta_P(\textbf{x})$函数，这与最小化原始问题是等价的，即
$$
\begin{equation}
\min_{\textbf{x}}{\theta_{P}(\textbf{x})} = \min_{\textbf{x}}{\max_{\alpha,\beta;\alpha^{(i)}\geq 0}L(\textbf{x},\alpha, \beta)}
\end{equation}
$$

则原始问题转变成了拉格朗日函数的极小极大问题，定义$p^{\star}=\min_{\textbf{x}}\theta_P(\textbf{x})$为原始问题的最优值。

#### 拉格朗日对偶问题
在上文中提到的拉格朗日函数的极小极大问题，求解的步骤是
1. 通过调节拉格朗日乘子$\alpha^{(i)}$和$\beta^{(j)}$使得$L(\textbf{x},\alpha, \beta)$最大化。
2. 通过在样本集中选择$\textbf{x}$使得$\theta_P(\textbf{x})$函数最小。

现在我们将两个步骤的顺序交换位置
1. 先对$L(\textbf{x},\alpha, \beta)$，通过调节$\textbf{x}$实现最小化，定义
$$
\begin{equation}
\theta_{D}(\alpha,\beta) = \min_{\textbf{x}}L(\textbf{x},\alpha, \beta)
\end{equation}
$$

2. 再考虑调节$\alpha$和$\beta$去最大化$\theta_{D}(\alpha,\beta)$函数，即
$$
\begin{equation}
\max_{\alpha,\beta;\alpha^{(i)}\geq 0}{\theta_{D}(\alpha,\beta)} = \max_{\alpha,\beta;\alpha^{(i)}\geq 0}{\min_{\textbf{x}}L(\textbf{x},\alpha, \beta)}
\end{equation}
$$

这是拉格朗日的极大极小问题。如果把第2步中要求$\alpha^{(i)}\geq 0$表示成约束条件，则极大极小问题可以写成约束最优化问题，即
$$
\begin{equation}
\max_{\alpha,\beta}{\theta_{D}(\alpha,\beta)} = \max_{\alpha,\beta}{\min_{\textbf{x}}L(\textbf{x},\alpha, \beta)} \\
\text{s.t} \quad \alpha^{(i)}\geq 0 \quad i=1,2,\cdots,k
\end{equation}
$$

这种写法称为原始问题的对偶问题。定义$d^{\star}=\max_{\alpha,\beta;\alpha^{(i)}\geq 0}{\theta_{D}(\alpha,\beta)}$为对偶问题的最优值。

#### 原始问题和对偶问题的关系
因为$\theta_D(\alpha, \beta)=\min_{\textbf{x}}L(\textbf{x},\alpha, \beta)$，通过调节$\textbf{x}$使得$L(\textbf{x},\alpha, \beta)$最小化。那么$\theta_D(\alpha, \beta)=\min_{\textbf{x}}L(\textbf{x},\alpha, \beta)\leq L(\textbf{x},\alpha, \beta)$

同时，$\theta_{P}(\textbf{x}) = \max_{\alpha,\beta;\alpha^{(i)}\geq 0}L(\textbf{x},\alpha, \beta)$，通过调节$\alpha$和$\beta$使得$L(\textbf{x},\alpha, \beta)$最大化。那么$\theta_{P}(\textbf{x}) = \max_{\alpha,\beta;\alpha^{(i)}\geq 0}L(\textbf{x},\alpha, \beta)\geq L(\textbf{x},\alpha, \beta)$

联立两式可得$\theta_D(\alpha, \beta) \leq \theta_{P}(\textbf{x})$。即$\theta_D$关于$\alpha$和$\beta$的函数不超过$\theta_P$关于$\textbf{x}$的函数。因此$\theta_D(\alpha, \beta)$的最大值也不会大于$\theta_{P}(\textbf{x})$的最小值。

所以$\max_{\alpha,\beta;\alpha^{(i)}\geq 0}\theta_D(\alpha, \beta) \leq \min_{\textbf{x}} \theta_P(\textbf{x})$，即有以下关系。

$$
\begin{equation}
d^{\star}=\max_{\alpha,\beta;\alpha^{(i)}\geq 0}{\min_{\textbf{x}}L(\textbf{x},\alpha, \beta)} \leq \min_{\textbf{x}}{\max_{\alpha,\beta;\alpha^{(i)}\geq 0}L(\textbf{x},\alpha, \beta)} = p^{\star}
\end{equation}
$$

说明了原始问题的最优值不小于对偶问题的最优值。由于我们要用求解对偶问题的最优值来求解原始问题的最优值，所以要保证$d^{\star}=p^{\star}$。接下来要介绍满足原始问题最优解等于对偶问题最优解的情况。

#### KKT条件
在原始问题和对偶问题中，$f(\textbf{x})$和$c^{(i)}(\textbf{x})$是凸函数，$h^{(j)}(\textbf{x})$是仿射函数。仿射函数的定义在这个链接可以找到。https://baike.baidu.com/item/%E4%BB%BF%E5%B0%84%E5%87%BD%E6%95%B0/9276178?fr=aladdin

不等式约束$c^{(i)}(\textbf{x})$是严格可行的，即存在$\textbf{x}$使得所有的$c^{(i)}(\textbf{x}) < 0 \quad i = 1,2,\cdots,k$。

则存在$x^{\star}$是原始问题的最优解，$\alpha^{\star}$和$\beta^{\star}$是对偶问题的最优解的充分必要条件是$x^{\star}$、$\alpha^{\star}$和$\beta^{\star}$要满足以下KKT条件
$$
\begin{equation}
\begin{aligned}
&\nabla_{\textbf{x}}L(\textbf{x}^{\star}, \alpha^{\star}, \beta^{\star}) = 0 \\
&\nabla_{\alpha}L(\textbf{x}^{\star}, \alpha^{\star}, \beta^{\star}) = 0 \\
&\nabla_{\beta}L(\textbf{x}^{\star}, \alpha^{\star}, \beta^{\star}) = 0 \\
&(\alpha^{(i)})^{\star}c^{(i)}(\textbf{x}^{\star})=0 \quad i = 1,2,\cdots,k \\
&c^{(i)}(\textbf{x}^{\star}) \leq 0 \quad i=1,2,\cdots, k \\
&(\alpha^{(i)})^{\star} \geq 0 \quad i=1,2,\cdots,k \\
& h^{(j)}(\textbf{x}^{\star})=0 \quad j=1,2,\cdots,l
\end{aligned}
\end{equation}
$$
其中，前三个条件是要求$f(\textbf{x})$，$c(\textbf{x})$和$h(\textbf{x})$在$\mathbb{R}^n$上是连续可微的，因此存在对$x^{\star}$、$\alpha^{\star}$和$\beta^{\star}$的偏导数，且偏导数为0。

第五和第七个条件是原始问题中的约束条件要求满足的。第六个条件是在引入拉格朗日时的拉格朗日乘子$\alpha^{(i)}, i=1,2,\cdots,k$要满足大于等于0的条件。第四个条件称为KKT对偶互补条件，可以看到$(\alpha^{(i)})^{\star}$和$c^{(i)}(\textbf{x}^{\star})$两项中至少要有一项等于0，在之后的推导中会利用到这个性质。

总结：在求解带约束的原始问题时，可以通过拉格朗日对偶性质，将其转换成对偶问题。通过求解对偶问题的最优解$\alpha^{\star}$和$\beta^{\star}$，反推原始问题的最优解$x^{\star}$，检查原始问题和对偶问题的最优解是否满足KKT条件。如果是，则$p^{\star}=d^{\star}=L(\textbf{x}^{\star}, \alpha^{\star},\beta^{\star})$

利用对偶问题来求解原始问题的知识介绍到这，接下来介绍怎样在最大化间隔问题上利用这个性质。

### 1.4 凸二次问题求解

#### 拉格朗日乘子法
回顾在1.2节的截尾提到的凸二次规划问题，我们称之为原始问题。利用1.3节介绍的拉格朗日对偶性质，可以把问题转变成对偶问题。
定义拉格朗日函数
$$
\begin{equation}
L(\textbf{w},b,\alpha) = \frac{1}{2}\|\textbf{w}\|^2 - \sum_{i=1}^N{\alpha^{(i)}y^{(i)}(\textbf{w}\cdot \textbf{x}^{(i)}+b)} + \sum_{i=1}^N{\alpha^{(i)}}
\end{equation}
$$
其中N个$\alpha^{(i)},i=1\cdots N$组成拉格朗日乘子向量$\alpha=(\alpha^{(1)},\alpha^{(2)},\cdots,\alpha^{(N)})$，即样本点的个数与拉格朗日乘子个数相同。

通过拉格朗日乘子，把N个不等式约束条件，转变到目标函数中的部分。得到了拉格朗日函数$L$。

利用拉格朗日对偶性质，原始问题的对偶问题是极大极小问题，即$\max_{\alpha;\alpha^{(i)} \geq 0}{\min_{\textbf{w}, b}{L(\textbf{w},b,\alpha)}}$，接下来分两步求解这个极大极小问题。

1. 求内层的$\min_{\textbf{w}, b}{L(\textbf{w},b,\alpha)}$

    将拉格朗日函数$L(\textbf{w},b,\alpha)$分别求对$\textbf{w}$和$b$的偏导数，令它们等于0，即
    $$
    \begin{equation}
    \begin{aligned}
    &\nabla_{\textbf{w}}L(\textbf{w},b,\alpha)=\textbf{w}-\sum_{i=1}^N{\alpha^{(i)}y^{(i)}\textbf{x}^{(i)}} = \textbf{0} \\
    &\nabla_{b}L(\textbf{w},b,\alpha) = \sum_{i=1}^N{\alpha^{(i)}y^{(i)}} = 0
    \end{aligned}
    \end{equation}
    $$
    第一个式子是对$\textbf{w}$向量求偏导数，等式右边是0向量，第二个式子是对标量$b$求偏导数，等式右边是数字0。整理上面两个式子可得
    $$
    \begin{equation}
    \begin{aligned}
    &\textbf{w}= \sum_{i=1}^N{\alpha^{(i)}y^{(i)}\textbf{x}^{(i)}} \\
    &\sum_{i=1}^N{\alpha^{(i)}y^{(i)}} = 0
    \end{aligned}
    \end{equation}
    $$
    将其代回拉格朗日函数。
    向量$\textbf{w}\in \mathbb{R}^n$。将$\textbf{w}$展开可以得到
    
    $$
    \begin{equation}
    \begin{aligned}
    \textbf{w} &= \{w_1,w_2,\cdots,w_n\} \\
    &=\sum_{i=1}^N{\alpha^{(i)}y^{(i)}\textbf{x}^{(i)}} \\
    &=\{\sum_{i=1}^N{\alpha^{(i)}y^{(i)}x^{(i)}_1}, \sum_{i=1}^N{\alpha^{(i)}y^{(i)}x^{(i)}_2,\cdots,\sum_{i=1}^N{\alpha^{(i)}y^{(i)}x^{(i)}_n}}\}
    \end{aligned}
    \end{equation}
    $$
    
    所以拉格朗日函数第一项中
    $$
    \begin{equation}
    \begin{aligned}
    \|\textbf{w}\|^2&=w_1^2 + w_2^2 + \cdots + w_n^2 \\
    &=(\sum_{i=1}^N{\alpha^{(i)}y^{(i)}x^{(i)}_1})^2 + (\sum_{i=1}^N{\alpha^{(i)}y^{(i)}x^{(i)}_2)^2 + \cdots + (\sum_{i=1}^N{\alpha^{(i)}y^{(i)}x^{(i)}_n}})^2 \\
    &=\sum_{i=1}^N{\sum_{j=1}^N{\alpha^{(i)}\alpha^{(j)}y^{(i)}y^{(j)}x_1^{(i)}x_1^{(j)}}} + \sum_{i=1}^N{\sum_{j=1}^N{\alpha^{(i)}\alpha^{(j)}y^{(i)}y^{(j)}x_2^{(i)}x_2^{(j)}}} + \cdots + \sum_{i=1}^N{\sum_{j=1}^N{\alpha^{(i)}\alpha^{(j)}y^{(i)}y^{(j)}x_n^{(i)}x_n^{(j)}}} \\
    &=\sum_{i=1}^N{\sum_{j=1}^N{\alpha^{(i)}\alpha^{(j)}y^{(i)}y^{(j)}(x_1^{(i)}x_1^{(j)} + x_2^{(i)}x_2^{(j)} + \cdots + x_n^{(i)}x_n^{(j)})}} \\
    &=\sum_{i=1}^N{\sum_{j=1}^N{\alpha^{(i)}\alpha^{(j)}y^{(i)}y^{(j)}(\textbf{x}^{(i)}\cdot \textbf{x}^{(j)})}}
    \end{aligned}
    \end{equation}
    $$
    
    拉格朗日函数的第二项
    
    $$
    \begin{equation}
    \begin{aligned}
    \sum_{i=1}^N{\alpha^{(i)}y^{(i)}(\textbf{w}\cdot \textbf{x}^{(i)}+b)} &=\sum_{i=1}^N{\alpha^{(i)}y^{(i)}((\sum_{j=1}^N{\alpha^{(j)}y^{(j)}\textbf{x}^{(j)}})\cdot \textbf{x}^{(i)}+b)} \\
    &=\sum_{i=1}^N{\sum_{j=1}^N{\alpha^{(i)}\alpha^{(j)}y^{(i)}y^{(j)}(\textbf{x}^{(i)}\cdot \textbf{x}^{(j)})}}
    \end{aligned}
    \end{equation}
    $$
    所以拉格朗日函数整理后可写成
    $$
    \begin{equation}
    \begin{aligned}
    L(\textbf{w},b,\alpha) &= \frac{1}{2}\sum_{i=1}^N{\sum_{j=1}^N{\alpha^{(i)}\alpha^{(j)}y^{(i)}y^{(j)}(\textbf{x}^{(i)}\cdot \textbf{x}^{(j)})}} - \sum_{i=1}^N{\sum_{j=1}^N{\alpha^{(i)}\alpha^{(j)}y^{(i)}y^{(j)}(\textbf{x}^{(i)}\cdot \textbf{x}^{(j)})}} + \sum_{i=1}^N{\alpha^{(i)}} \\
    &= -\frac{1}{2}\sum_{i=1}^N{\sum_{j=1}^N{\alpha^{(i)}\alpha^{(j)}y^{(i)}y^{(j)}(\textbf{x}^{(i)}\cdot \textbf{x}^{(j)})}} + \sum_{i=1}^N{\alpha^{(i)}}
    \end{aligned}
    \end{equation}
    $$
    
    此时得到的拉格朗日函数为最小值，即
    $$
    \begin{equation}
    \begin{aligned}
    \min_{\textbf{w}, b}{L(\textbf{w},b,\alpha)} &= 
    -\frac{1}{2}\sum_{i=1}^N{\sum_{j=1}^N{\alpha^{(i)}\alpha^{(j)}y^{(i)}y^{(j)}(\textbf{x}^{(i)}\cdot \textbf{x}^{(j)})}} + \sum_{i=1}^N{\alpha^{(i)}}
    \end{aligned}
    \end{equation}
    $$
    在这个表达式中，$\textbf{w}$和$b$消失了，因为它们被$\alpha$、$y$和$\textbf{x}$表示了。

2. 求$\min_{\textbf{w}, b}{L(\textbf{w},b,\alpha)}$关于$\alpha$的极大值。
    
    回顾1.3节，定义$\theta_D(\alpha) = \min_{\textbf{w}, b}{L(\textbf{w},b,\alpha)}$，对$\theta_D(\alpha)$求关于$\alpha$的极大值
    $$
    \begin{equation}
    \begin{aligned}
    \max_{\alpha}\quad &{-\frac{1}{2}\sum_{i=1}^N{\sum_{j=1}^N{\alpha^{(i)}\alpha^{(j)}y^{(i)}y^{(j)}(\textbf{x}^{(i)}\cdot \textbf{x}^{(j)})}} + \sum_{i=1}^N{\alpha^{(i)}}} \\
    \text{s. t. } \quad & \sum_{i=1}^N{\alpha^{(i)}y^{(i)}} = 0 \\
    \quad & \alpha^{(i)} \geq 0, i=1,2,\cdots,N
    \end{aligned}
    \end{equation}
    $$
    如果把负号拿掉，则目标函数变成
    $$
    \begin{equation}
    \begin{aligned}
    \min_{\alpha}\quad &{\frac{1}{2}\sum_{i=1}^N{\sum_{j=1}^N{\alpha^{(i)}\alpha^{(j)}y^{(i)}y^{(j)}(\textbf{x}^{(i)}\cdot \textbf{x}^{(j)})}} - \sum_{i=1}^N{\alpha^{(i)}}} \\
    \text{s. t. } \quad & \sum_{i=1}^N{\alpha^{(i)}y^{(i)}} = 0 \\
    \quad & \alpha^{(i)} \geq 0, i=1,2,\cdots,N
    \end{aligned}
    \end{equation}
    $$
    
    在原始问题\ref{primal}中，目标函数$\frac{1}{2}\|\textbf{w}\|^2$和约束条件$y^{(i)}\left(\textbf{w} \cdot \textbf{x} + b\right) -1$是凸函数，且$y^{(i)}\left(\textbf{w} \cdot \textbf{x} + b\right) -1$严格可行，则存在$\textbf{w}^{\star}$和$b^{\star}$是原始问题的解，$\alpha^{\star}$是对偶问题的解。下面介绍$(\textbf{w}^{\star}, b^{\star})$和$\alpha^{\star}$的关系。
    

#### 原始问题与对偶问题的解的关系

我们通过求解对偶问题得到了最优解$\alpha^{\star} = \{(\alpha^{(1)})^{\star}, (\alpha^{(2)})^{\star}, \cdots, (\alpha^{(N)})^{\star}\}^T \in \mathbb{R}^N$。根据KKT条件可以推导出$\alpha^{\star}$与$(\textbf{w}^{\star}, b^{\star})$的关系。

$$
\begin{equation}
\begin{aligned}
&\nabla_{\textbf{w}}L(\textbf{w}^{\star}, b^{\star}, \alpha^{\star})=\textbf{w}^{\star} - \sum_{i=1}^N{(\alpha^{(i)})^{\star}y^{(i)}\textbf{x}^{(i)}}=0 \\
&\nabla_{b}L(\textbf{w}^{\star}, b^{\star}, \alpha^{\star}) = -\sum_{i=1}^N{(\alpha^{(i)})^{\star}y^{(i)}} = 0 \\
&(\alpha^{(i)})^{\star}(y^{(i)}(\textbf{w}^{\star}\cdot \textbf{x}^{(i)} + b^{\star}) - 1) = 0, i = 1, 2, \cdots, N \\
&y^{(i)}(\textbf{w}^{\star}\cdot \textbf{x}^{(i)} + b^{\star}) - 1 \geq 0, i = 1, 2, \cdots, N \\
&(\alpha^{(i)})^{\star} \geq 0, i = 1, 2, \cdots, N
\end{aligned}
\end{equation}
$$

前两个条件是令偏导数等于0得到的解，代回偏导数满足条件。第三个是KKT对偶互补条件。第四个条件是原始问题中的约束条件。最后一个是拉格朗日乘子要满足的条件。

由第一个条件可得
$$
\begin{equation}
\textbf{w}^{\star} = \sum_{i=1}^N{(\alpha^{(i)})^{\star}y^{(i)}\textbf{x}^{(i)}}
\label{wstar}
\end{equation}
$$

可以看到$\textbf{w}^{\star}$是由N个含有拉格朗日乘子$(\alpha^{(i)})^{\star}$的项相加得到。如果所有的拉格朗日乘子都等于0，则$\textbf{w}^{\star} = \textbf{0}$，而0向量不是原始问题的解，说明不可能所有的$(\alpha^{(i)})^{\star}$都等于0。

假设存在$(\alpha^{(j)})^{\star} > 0$，由KKT条件中的对偶互补条件可得，
$$
\begin{equation}
y^{(j)}(\textbf{w}^{\star}\cdot \textbf{x}^{(j)} + b^{\star}) - 1 = 0
\end{equation}
$$

表明第j个样本点使得原始问题的不等式约束取到等号。

两边同时乘以$y^{(j)}$得到$(y^{(j)})^2(\textbf{w}^{\star}\cdot \textbf{x}^{(j)} + b^{\star})=y^{(j)}$。因为$(y^{(j)})^2=1$，
$$
\begin{equation}
\textbf{w}^{\star}\cdot \textbf{x}^{(j)} + b^{\star}=y^{(j)}
\label{complement}
\end{equation}
$$

将$\ref{wstar}$带入$\ref{complement}$，整理后得到
$$
\begin{equation}
\begin{aligned}
b^{\star} &= y^{(j)} - \textbf{w}^{\star}\cdot \textbf{x}^{(j)} \\
&= y^{(j)} - \sum_{i=1}^N{(\alpha^{(i)})^{\star}y^{(i)}(\textbf{x}^{(i)}}\cdot \textbf{x}^{(j)})
\end{aligned}
\label{bstar}
\end{equation}
$$

根据$\ref{wstar}$和$\ref{bstar}$可知，我们可以用求解对偶问题得到的解$\alpha^{\star}$去表达原始问题的解($\textbf{w}^{\star}, b^{\star}$)。根据$\ref{classifier}$可以把分离超平面表示成
$$
\begin{equation}
\sum_{i=1}^N{(\alpha^{(i)})^{\star}y^{(i)}(\textbf{x}^{(i)}} \cdot \textbf{x}) + b^{\star} = 0
\end{equation}
$$

这是线性可分的SVM的对偶形式。可以看到分离超平面依赖于训练样本$\textbf{x}^{(j)}$和输入的$\textbf{x}$的内积。
类别决策函数则为将样本点带入分类超平面的方程式后得到的结果的符号，写成
$$
\begin{equation}
f(\textbf{x})=\text{sign}(\sum_{i=1}^N{(\alpha^{(i)})^{\star}y^{(i)}(\textbf{x}^{(i)}} \cdot \textbf{x}) + b^{\star})
\end{equation}
$$

根据$\ref{wstar}$可知，分类超平面只与$(\alpha^{(i)})^{\star}>0, i=1,2,\cdots,N$对应的样本点($\textbf{x}^{(i)}$, $y^{(i)}$)有关，而$\alpha^{(i)})^{\star}=0$对应的样本点对($\textbf{w}^{\star}$, $b^{\star}$)没有贡献。我们称$(\alpha^{(i)})^{\star}>0$为支撑向量。

其实很好理解，根据KKT对偶互补条件可知，$(\alpha^{(i)})^{\star}>0$，则$y^{(i)}(\textbf{w}^{\star}\cdot \textbf{x}^{(i)} + b^{\star})=0$，则$\textbf{w}^{\star}\cdot \textbf{x}^{(i)} + b^{\star} =\pm 1$，表明该样本点距离分离超平面的距离等于1，即点在分离超平面上。

以上是有关线性可分支撑向量机的内容。

## 2 线性不可分支撑向量机及其对偶问题
### 2.1 问题描述
有关数据集$T$以及$\textbf{x}$和$y$的定义和1.1节一致。区别在于，此时的数据集$T$无法做到线性可分。此时数据集$T$中存在样本点无法满足$y^{(i)}\left(\textbf{w} \cdot \textbf{x}^{(i)} + b\right) -1 \geq 0$。此时为每个样本点引入松弛变量$\xi^{(i)}$。将约束条件改写成
$$
\begin{equation}
y^{(i)}\left(\textbf{w} \cdot \textbf{x}^{(i)} + b\right)  \geq 1 - \xi^{(i)}
\end{equation}
$$
同时修改目标函数，使得目标函数变为
$$
\begin{equation}
\frac{1}{2}\|\textbf{w}\|^2 + C\sum_{i=1}^N{\xi^{(i)}}
\end{equation}
$$
其中$C > 0$作为超参数(hyperparameter)输入，后面一项为惩罚项。C越大表明对惩罚越严重，反之对惩罚越小。

此时的凸二次规划问题为
$$
\begin{equation}
\begin{aligned}
\min_{\textbf{w}, b, \xi} \quad &\frac{1}{2}\|\textbf{w}\|^2 + C\sum_{i=1}^N{\xi^{(i)}}\\
\text{s. t. } \quad &y^{(i)}\left(\textbf{w} \cdot \textbf{x}^{(i)} + b\right) \geq 1-\xi^{(i)}, \quad i=1,2,\cdots, N\\
&\xi^{(i)} \geq 0, \quad i=1,2,\cdots, N
\end{aligned}
\end{equation}
$$

这是一个目标函数为二次函数，约束条件为一次的凸二次规划问题。通过求解该问题得到$(\textbf{w}^{\star}, b^{\star})$，则最优超平面为$\textbf{w}^{\star}\cdot \textbf{x} + b^{\star}=0$

### 2.2 凸二次问题求解
在2.1节结尾的最小化问题是原始问题，我们依旧可以用在第1.3节中介绍的拉格朗日对偶性质将其转换成对偶问题来求解。定义拉格朗日函数
$$
\begin{equation}
L(\textbf{w},b,\xi, \alpha, \mu) = \frac{1}{2}\|\textbf{w}\|^2 + C\sum_{i=1}^N{\xi^{(i)}} - \sum_{i=1}^N{\alpha^{(i)}(y^{(i)}(\textbf{w}\cdot \textbf{x}^{(i)}+b) - 1 + \xi^{(i)})} - \sum_{i=1}^N{\mu^{(i)}\xi^{(i)}}
\end{equation}
$$
其中拉格朗日乘子$\alpha^{(i)}\geq 0,\xi^{(i)} \geq 0, i=1,2,\cdots,N$
根据1.4节，分两步走求解该原始问题的对偶问题，即极大极小问题。

首先求$L(\textbf{w},b,\xi, \alpha, \mu)$对$\textbf{w},b,\xi$求极小，它们的偏导数分别是

$$
\begin{equation}
\begin{aligned}
&\nabla_{\textbf{w}}L(\textbf{w},b,\xi, \alpha, \mu) = \textbf{w} - \sum_{i=1}^{N}{\alpha^{(i)}y^{(i)}\textbf{x}^{(i)}}=0 \\
&\nabla_{b}L(\textbf{w},b,\xi, \alpha, \mu) = -\sum_{i=1}^N{\alpha^{(i)}y^{(i)}} = 0 \\
&\nabla_{\xi^{(i)}}L(\textbf{w},b,\xi, \alpha, \mu) = C - \alpha^{(i)} - \mu^{(i)} = 0, \quad i=1,2,\cdots,N
\end{aligned}
\end{equation}
$$

整理得
$$
\begin{equation}
\begin{aligned}
&\textbf{w} = \sum_{i=1}^{N}{\alpha^{(i)}y^{(i)}\textbf{x}^{(i)}} \\
&\sum_{i=1}^N{\alpha^{(i)}y^{(i)}} = 0 \\
&C - \alpha^{(i)} - \mu^{(i)} = 0, \quad i=1,2,\cdots,N
\end{aligned}
\end{equation}
$$

代回拉格朗日函数，整理得
$$
\begin{equation}
\begin{aligned}
\min_{\textbf{w}, b, \xi}{L(\textbf{w},b,\xi, \alpha, \mu)} &= 
-\frac{1}{2}\sum_{i=1}^N{\sum_{j=1}^N{\alpha^{(i)}\alpha^{(j)}y^{(i)}y^{(j)}(\textbf{x}^{(i)}\cdot \textbf{x}^{(j)})}} + \sum_{i=1}^N{\alpha^{(i)}}
\end{aligned}
\end{equation}
$$
整理的过程和1.4节中的一致。
将$\min_{\textbf{w}, b, \xi}{L(\textbf{w},b,\xi, \alpha, \mu)}$定义成$\theta_D(\alpha, \mu)$，求$\theta_D(\alpha, \mu)$对$\alpha$的极大
$$
\begin{equation}
\begin{aligned}
\max_{\alpha}\quad &{-\frac{1}{2}\sum_{i=1}^N{\sum_{j=1}^N{\alpha^{(i)}\alpha^{(j)}y^{(i)}y^{(j)}(\textbf{x}^{(i)}\cdot \textbf{x}^{(j)})}} + \sum_{i=1}^N{\alpha^{(i)}}} \\
\text{s. t. } \quad & \sum_{i=1}^N{\alpha^{(i)}y^{(i)}} = 0 \\
\quad & \alpha^{(i)} \geq 0, \quad i=1,2,\cdots,N\\
\quad & \mu^{(i)} \geq 0, \quad i=1,2,\cdots,N\\
\quad & C - \alpha^{(i)} - \mu^{(i)} = 0, \quad i=1,2,\cdots,N
\end{aligned}
\end{equation}
$$

可以把$\alpha^{(i)}, \mu^{(i)}, C$的约束条件改写成$0\leq \alpha^{(i)} \leq C$。目标函数中添加一个负号，最大化问题改写成最小化问题，于是变成了如下的对偶问题的形式
$$
\begin{equation}
\begin{aligned}
\min_{\alpha}\quad &{\frac{1}{2}\sum_{i=1}^N{\sum_{j=1}^N{\alpha^{(i)}\alpha^{(j)}y^{(i)}y^{(j)}(\textbf{x}^{(i)}\cdot \textbf{x}^{(j)})}} - \sum_{i=1}^N{\alpha^{(i)}}} \\
\text{s. t. } \quad & \sum_{i=1}^N{\alpha^{(i)}y^{(i)}} = 0 \\
\quad & C \geq \alpha^{(i)} \geq 0, i=1,2,\cdots,N
\end{aligned}
\end{equation}
$$

我们通过求解对偶问题得到了最优解$\alpha^{\star} = \{(\alpha^{(1)})^{\star}, (\alpha^{(2)})^{\star}, \cdots, (\alpha^{(N)})^{\star}\}^T \in \mathbb{R}^N$。根据KKT条件可以推导出$\alpha^{\star}$与$(\textbf{w}^{\star}, b^{\star})$的关系。
$$
\begin{equation}
\begin{aligned}
&\nabla_{\textbf{w}}L(\textbf{w}^{\star}, b^{\star}, \xi^{\star}, \alpha^{\star}, \mu^{\star})=\textbf{w}^{\star} - \sum_{i=1}^N{(\alpha^{(i)})^{\star}y^{(i)}\textbf{x}^{(i)}}=0 \\
&\nabla_{b}L(\textbf{w}^{\star}, b^{\star}, \xi^{\star}, \alpha^{\star}, \mu^{\star}) = -\sum_{i=1}^N{(\alpha^{(i)})^{\star}y^{(i)}} = 0 \\
& \nabla_{\xi}L(\textbf{w},b,\xi, \alpha, \mu) = C - \alpha^{\star} - \mu^{\star} = 0\\
&(\alpha^{(i)})^{\star}(y^{(i)}(\textbf{w}^{\star}\cdot \textbf{x}^{(i)} + b^{\star}) - 1 + (\xi^{(i)})^{\star}) = 0, \quad  i = 1, 2, \cdots, N \\
&(\mu^{(i)})^{\star}(\xi^{(i)})^{\star} = 0 \\
&y^{(i)}(\textbf{w}^{\star}\cdot \textbf{x}^{(i)} + b^{\star}) - 1 + (\xi^{(i)})^{\star} \geq 0, \quad i = 1, 2, \cdots, N \\
&(\xi^{(i)})^{\star} \geq 0, \quad i = 1, 2, \cdots, N \\
&(\alpha^{(i)})^{\star} \geq 0, \quad i = 1, 2, \cdots, N \\
&(\mu^{(i)})^{\star} \geq 0, \quad i = 1, 2, \cdots, N \\
\end{aligned}
\end{equation}
$$

其中前三个条件为求拉格朗日函数对$\textbf{w}, b, \xi$的偏导数等于0得到的。第四和第五个函数分别来自原始问题中的两个约束条件，通过引入拉格朗日乘子$\alpha$和$\mu$引入到目标函数时，要满足的KKT对偶互补条件。第六和第七个条件是原始问题的约束条件。最后两个条件为引入的拉格朗日乘子需要满足的条件。

通过整理这些KKT条件可得

$$
\begin{equation}
\begin{aligned}
&\textbf{w}^{\star} = \sum_{i=1}^N{(\alpha^{(i)})^{\star}y^{(i)}\textbf{x}^{(i)}} \\
&b^{\star} = y^{(j)} - \sum_{i=1}^N{(\alpha^{(i)})^{\star}y^{(i)}(\textbf{x}^{(i)}}\cdot \textbf{x}^{(j)})
\end{aligned}
\end{equation}
$$

可以看到原始问题的最优解$(\textbf{w}^{\star}, b^{\star})$通过拉格朗日乘子$\alpha^{\star}$表示出来了。

根据$\ref{classifier}$可以把分离超平面表示成
$$
\begin{equation}
\sum_{i=1}^N{(\alpha^{(i)})^{\star}y^{(i)}(\textbf{x}^{(i)}} \cdot \textbf{x}) + b^{\star} = 0
\end{equation}
$$

这是线性可分的SVM的对偶形式。可以看到分离超平面依赖于训练样本$\textbf{x}^{(j)}$和输入的$\textbf{x}$的内积。
类别决策函数则为将样本点带入分类超平面的方程式后得到的结果的符号，写成
$$
\begin{equation}
f(\textbf{x})=\text{sign}(\sum_{i=1}^N{(\alpha^{(i)})^{\star}y^{(i)}(\textbf{x}^{(i)}} \cdot \textbf{x}) + b^{\star})
\end{equation}
$$

注意：在利用$b^{\star} = y^{(j)} - \sum_{i=1}^N{(\alpha^{(i)})^{\star}y^{(i)}(\textbf{x}^{(i)}}\cdot \textbf{x}^{(j)})$求解$b^{\star}$时，要在$\alpha^{\star}=\{(\alpha^{(1)})^{\star},(\alpha^{(N)})^{\star},\cdots,(\alpha^{(N)})^{\star}\}$中选择$0 < (\alpha^{(j)})^{\star} < C$，则可以求出多个$b^{\star}$。在实际情况中，$b^{\star}$可以取平均值。

### 2.3 支撑向量

<img src="img/软间隔分类样例.png">
图中画o的点，有在间隔边界上，有在分离超平面与间隔之间，有在分离超平面上，也有误分在分离超平面另外一侧。这些点都会影响分离超平面的选择，按照定义，它们被称为支撑向量。支撑向量的特点是，对应的拉格朗日乘子$\alpha^{(i)} > 0$。

若$\alpha^{(i)} < C$，则根据KKT的第三和第五个条件可知$\xi^{(i)}=0$。说明这个样本点不需要做间隔的松弛，即它在分类超平面的间隔上。
若$\alpha^{(i)} = C$，则根据KKT的第三个条件可知$\mu^{(i)}=0$，则第五个条件告知$\xi^{(i)}$要分情况讨论。
$\xi^{(i)}$在引入的时候是以满足$\xi^{(i)} \geq 0$为约束的。

当$\xi^{(i)}=0$表示样本点不需要做间隔松弛，即它在分离超平面的间隔上。

当$1 > \xi^{(i)} > 0$表示样本点在分离超平面和间隔之间。

当$\xi^{(i)} = 1$表示样本点在分离超平面上。

当$\xi^{(i)} > 1$表示样本点在跨过了分离超平面，跑到了错误的一侧。

## 3. 核函数
对于数据集无法利用线性模型进行划分的，我们要使用非线性支撑向量机，通过核技巧(kernel trick)来实现从线性模型向非线性模型转变。使得在变换前需要用一个分离超曲面能够分开的数据集，在变换后，可以用一个分离超平面来分离数据。

### 3.1 问题描述
给定一组训练数据集
$$
\begin{equation}
T=\{(\textbf{x}^{(1)},y^{(1)}),(\textbf{x}^{(2)},y^{(2)}),\cdots, (\textbf{x}^{(N)}, y^{(N)})\}
\end{equation}
$$
该数据集包含N个样本点。每个样本点的$\textbf{x}$加粗显示，表示这是一个向量，$\textbf{x}\in \mathbb{R}^{n}$，当然如果n=1，则$\textbf{x}$是一个标量。

在$\textbf{x}^{(i)}=(x^{(i)}_1,x^{(i)}_2,\cdots, x^{(i)}_n)$中的每一个维度，表示该样本点的一个特征，样本集中的每个样本点有n个维度或特征。

$y^{(i)}$表示第i个样本点的类别，$y\in\{+1, -1\}$，当$y^{(i)}=1$，则表示$x^{(i)}$是正例。当$y^{(i)}=-1$，则表示$x^{(i)}$是负例。

<img src = img/非线性分类问题与核技巧示例.png>

左图表示原始空间，由二维特征$(x_1,x_2)$表述。图中包含一个非线性分类的数据集，无法用一个线性模型很好的划分它们。图中需要用一个椭圆来划分数据集。在原始的用$(x_1,x_2)$描述的空间中，椭圆的方程式为
$$
\begin{equation}
w_1x_1^2 + w_2x_2^2 + b = 0
\end{equation}
$$

一般的做法是将左图中的原始空间，变换到右图中的映射空间。映射空间由变换后的二维特征$(z_1,z_2)$表述，从而使得原始空间中的每个点，在映射空间中都有一一对应。在映射空间中，o和x的数据可以被一个线性模型划分。

具体来看，原始空间为$\mathcal{X} \subset \mathbb{R}^2$，样本点$\textbf{x}=(x_1, x_2)^T \in \mathcal{X}$。映射空间$\mathcal{Z} \subset \mathbb{R}^2$，映射后的样本点$\textbf{z}=(z_1, z_2)^T \in \mathcal{Z}$。

定义映射函数$\phi(\textbf{x}): \textbf{x}\rightarrow\textbf{x}^2$，即$\textbf{z}=\textbf{x}^2=(x_1^2, x_2^2)$。这样在原始空间中的椭圆的表达式变成由$\textbf{z}$来描述
$$
\begin{equation}
w_1z_1 + w_2z_2 + b = 0
\end{equation}
$$

可以看到在映射空间中的分离方程式是线性的，此时变成一个线性可分问题。

### 3.2 核函数
对于以上非线性可分的问题求解步骤是，先把数据集所在的原始空间，经过一次非线性变换，变换成映射空间。然后在映射空间中做线性可分的支撑向量机求解过程。回顾支撑向量机求解过程，以第一节介绍的线性可分支撑向量机为例，
分离超平面经过一番推导后可以用以下式子来表达
$$
\begin{equation}
f(\textbf{x})=\sum_{i=1}^N{(\alpha^{(i)})^{\star}y^{(i)}(\textbf{x}^{(i)}} \cdot \textbf{x}) + b^{\star} = 0
\end{equation}
$$
第一项中涉及到向量内积$\textbf{x}^{(i)}\cdot \textbf{x}$。
假如说现在是一个非线性可分问题，按照在本节一开始的求解过程，我们现在要求的是一个分离超曲面的函数，要把数据集转变到映射空间中，那么假设映射函数为$\phi(\textbf{x})$，则分离函数可以表达成
$$
\begin{equation}
f(\textbf{x})=\sum_{i=1}^N{(\alpha^{(i)})^{\star}y^{(i)}(\phi(\textbf{x}^{(i)}}) \cdot \phi(\textbf{x})) + b^{\star} = 0
\end{equation}
$$

但是上面的求解过程，在实际的工程实现上，存在效率的问题。接下来定义核函数。
$$
K(\textbf{x}^{(i)}, \textbf{x}^{(j)}) = \phi(\textbf{x}^{(i)}) \cdot \phi(\textbf{x}^{(j)})
$$
称$K(\textbf{x}^{(i)}, \textbf{x}^{(j)})$为核函数。

除了存在计算效率的问题，有些情况是只使用映射函数$\phi(\textbf{x})$无法解决的，例如在映射过后要求数据处于一个无穷多维度的空间，这种情况无法显示的写出映射函数$\phi(\textbf{x})$来。后面会具体提到这种情况。

有了核函数的定义，则支撑向量机的分离函数可以写成
$$
\begin{equation}
f(\textbf{x})=\sum_{i=1}^N{(\alpha^{(i)})^{\star}y^{(i)}K(\textbf{x}^{(i)}} ,\textbf{x}) + b^{\star} = 0
\end{equation}
$$

$b^{\star}$可以写成
$$
\begin{equation}
\begin{aligned}
b^{\star} &= y^{(j)} - \sum_{i=1}^N{(\alpha^{(i)})^{\star}y^{(i)}K(\textbf{x}^{(i)}}, \textbf{x}^{(j)})
\end{aligned}
\end{equation}
$$
对于有间隔松弛的场景，选定适当的参数$C$，套用上述的表达式即可。

关于核函数的性质，下面应用《统计学习方法》上的一个例题描述。
对于一个给定的核函数$K(\textbf{x}^{(i)}, \textbf{x}^{(j)})$，其表达的映射函数$\phi(\textbf{x})$和映射后的空间不是唯一的。用以下例子说明。

假设原始空间是二维的$\mathbb{R}^2$，核函数$K(\textbf{x}, \textbf{z})=(\textbf{x} \cdot \textbf{z})^2$。那么有以下的$\phi(\cdot)$和映射空间对。

因为原始空间是二维的，所以把输入的$\textbf{x}$和$\textbf{z}$分别记做$(x_1,x_2)^T$和$(z_1,z_2)^T$。（加向量转置是因为数学上习惯用列向量表达）
1. 取映射空间是三维的$\mathbb{R}^3$，因为
    $$
    K(\textbf{x}, \textbf{z}) = (\textbf{x}\cdot \textbf{z})^2=(x_1z_1+x_2z_2)^2 = (x_1z_1)^2 + 2x_1z_1x_2z_2 + (x_2z_2)^2
    $$
    所以此时可以取映射函数$\phi(\textbf{x})=\phi(x_1,x_2)=(x_1^2, \sqrt{2}x_1x_2, x_2^2)^T$。利用该映射函数可以验证
    $$
    \phi(x_1,x_2) \cdot \phi(z_1,z_2) = (x_1^2, \sqrt{2}x_1x_2, x_2^2) \cdot (z_1^2, \sqrt{2}z_1z_2, z_2^2) = x_1^2z_1^2 + 2x_1x_2z_1z_2 + x_2^2z_2^2 = K(\textbf{x}, \textbf{z})
    $$
2. 取映射空间是三维的$\mathbb{R}^3$，同时取映射函数$\phi(\textbf{x})=\phi(x_1,x_2)=\frac{1}{\sqrt{2}}(x_1^2-x_2^2, 2x_1x_2, x_1^2+x_2^2)^T$。利用该映射函数可以验证
   
    $$
    \begin{equation}
    \begin{aligned}
    \phi(x_1,x_2) \cdot \phi(z_1,z_2) &= \frac{1}{\sqrt{2}}(x_1^2-x_2^2, 2x_1x_2, x_1^2+x_2^2) \cdot \frac{1}{\sqrt{2}}(z_1^2-z_2^2, 2z_1z_2, z_1^2+z_2^2)  \\
    &= \frac{1}{2}(x_1^2z_1^2 - x_1^2z_2^2 - x_2^2z_1^2 + x_2^2z_2^2 +  x_1^2z_1^2 + x_1^2z_2^2 + x_2^2z_1^2 + x_2^2z_2^2 + 4x_1x_2z_1z_2) \\
    &= x_1^2z_1^2 + 2x_1x_2z_1z_2 + x_2^2z_2^2 \\
    &= K(\textbf{x}, \textbf{z})
    \end{aligned}
    \end{equation}
    $$
    
3. 取映射空间是四维的$\mathbb{R}^4$，同时取映射函数$\phi(\textbf{x})=\phi(x_1,x_2)=(x_1^2, x_1x_2, x_1x_2, x_2^2)^T$，利用该映射函数可以验证
    $$
    \begin{equation}
    \begin{aligned}
    \phi(x_1,x_2) \cdot \phi(z_1,z_2) &= (x_1^2, x_1x_2, x_1x_2, x_2^2) \cdot (z_1^2, z_1z_2, z_1z_2, z_2^2)  \\
    &= x_1^2z_1^2 + x_1x_2z_1z_2 + x_1x_2z_1z_2 + x_2^2z_2^2 \\
    &= x_1^2z_1^2 + 2x_1x_2z_1z_2 + x_2^2z_2^2 \\
    &= K(\textbf{x}, \textbf{z})
    \end{aligned}
    \end{equation}
    $$

### 3.3 核函数的种类
常见的核函数种类有多项式核函数、高斯核函数、字符串核函数等
1. 多项式核函数
    $$
    \begin{equation}
    K(\textbf{x}, \textbf{z}) = (\textbf{x} \cdot \textbf{z} + 1)^p
    \end{equation}
    $$
    当$p=1$时，也称为线性核函数
    
2. 高斯核函数
    $$
    \begin{equation}
    K(\textbf{x}, \textbf{z}) = \exp{\left(-\frac{\|\textbf{x}-\textbf{z}\|}{2\sigma^2}\right)}
     \end{equation}
    $$
    此时的支撑向量机是高斯径向基函数(radial basis function)分类器。
    
字符串核函数不作为本文的重点，具体可以参考《统计学习方法》第122页

## 4. SMO序列最小化优化算法
回顾第1或第2节求解分离超平面时，在求解以$\alpha$向量为变量凸二次规划问题时，我们都是假设通过某种方法找到了$\alpha^{\star} = \{(\alpha^{(1)})^{\star}, (\alpha^{(2)})^{\star}, \cdots, (\alpha^{(N)})^{\star}\}^T \in \mathbb{R}^N$。可以看到，$\alpha \in \mathbb{R}^N$是一个$N$维的向量，$N$表示训练样本点的个数。训练样本点的个数很大时，一般求解二次规划问题的方法的性能会比较低。SMO算法是一种快速求解$\alpha^{\star}$的算法。

### 4.1 算法思路
SMO算法的基本思路：判断一个$\alpha$向量是否为找到的对偶问题的最优解时，要看它是否满足KKT条件，这是一个充分必要条件。KKT条件是要求$\alpha$向量中的每一个元素$\alpha^{(i)}$都满足条件。如果不满足KKT条件，就说明当前的$\alpha$还不是最优解。此时，选择$\alpha$向量中的两个维度，固定其他维度。针对这两个维度来构建二次规划问题，称为子问题。因为子问题仍然是是二次规划问题，找到的解为全局最优解，所以即使只允许两个维度的变量进行调整，找到的解也要比当前的解要更优、更加接近全局最优解。重要的是，子问题可以通过解析方法来求解，这可以大大提高整个问题的求解速度。选择两个维度的变量的方法是，选择违反KKT条件最严重的一个，另外一个由约束条件确定。在KKT条件中，$\alpha$向量需要满足
$$
\quad \sum_{i=1}^N{\alpha^{(i)}y^{(i)}} = 0
$$
因此我们确定的两个维度，实际上只有一个维度是自由变量。假设$\alpha^{(1)},\alpha^{(2)}$是选定的两个变量，当通过求解二次规划问题得到了$\alpha^{(2)}$后，根据$\alpha^{(1)}=-y^{(1)}\sum_{i=2}^N{\alpha^{(i)}y^{(i)}}$同时可以确定$\alpha^{(1)}$，这样就同时把$\alpha^{(1)},\alpha^{(2)}$更新了。循环这个过程。

注意，这个算法是一个启发式算法，即最后找到的解可能不是$\alpha^{\star}$，而是一个近似解$\hat{\alpha}$，需要在算法开始前设定一个精度$\epsilon$。

下面会解释一下怎样求解两个维度的变量的二次规划问题，以及具体怎样选出两个维度的变量。

### 4.2 $\alpha^{(1)}$和$\alpha^{(2)}$的二次规划求解方法
不失一般性，假设选择$\alpha^{(1)}$和$\alpha^{(2)}$，其余变量固定，标记函数名为$W(\alpha^{(1)}, \alpha^{(2)})$，则对偶问题
$$
\begin{equation}
\begin{aligned}
\min_{\alpha}\quad &{\frac{1}{2}\sum_{i=1}^N{\sum_{j=1}^N{\alpha^{(i)}\alpha^{(j)}y^{(i)}y^{(j)}(\textbf{x}^{(i)}\cdot \textbf{x}^{(j)})}} - \sum_{i=1}^N{\alpha^{(i)}}} \\
\text{s. t. } \quad & \sum_{i=1}^N{\alpha^{(i)}y^{(i)}} = 0 \\
\quad & C \geq \alpha^{(i)} \geq 0, i=1,2,\cdots,N
\end{aligned}
\end{equation}
$$
可以写成
$$
\begin{equation}
\begin{aligned}
\min_{\alpha^{(1)}, \alpha^{(2)}} \quad W(\alpha^{(1)}, \alpha^{(2)}) &={\frac{1}{2}\sum_{i=1}^N{\sum_{j=1}^N{\alpha^{(i)}\alpha^{(j)}y^{(i)}y^{(j)}(\textbf{x}^{(i)}\cdot \textbf{x}^{(j)})}} - \sum_{i=1}^N{\alpha^{(i)}}} \\
&= \frac{1}{2}K(\alpha^{(1)}, \alpha^{(1)}) (y^{(1)})^2 (\alpha^{(1)})^2 + \frac{1}{2}K(\alpha^{(2)}, \alpha^{(2)}) (y^{(2)})^2 (\alpha^{(2)})^2  + y^{(1)}y^{(2)}K(\alpha^{(1)}, \alpha^{(2)}) \alpha^{(1)}\alpha^{(2)}  \\
&- \alpha^{(1)} - \alpha^{(2)} + y^{(1)}\alpha^{(1)}\sum_{i=3}^N{y^{(i)}\alpha^{(i)}K(\alpha^{(i)}, \alpha^{(1)})} + y^{(2)}\alpha^{(2)}\sum_{i=3}^N{y^{(i)}\alpha^{(i)}K(\alpha^{(i)}, \alpha^{(2)})}\\
\text{s. t. } & \alpha^{(1)}y^{(1)} +  \alpha^{(2)}y^{(2)} = -\sum_{i=3}^N{\alpha^{(i)}y^{(i)}} = \zeta \\
\quad & C \geq \alpha^{(i)} \geq 0, i=1,2,\cdots,N
\end{aligned}
\end{equation}
$$

因为在这个问题中的变量仅有$\alpha^{(1)}$和$\alpha^{(2)}$，其余不可调，因此把和$\alpha^{(1)}$和$\alpha^{(2)}$无关的项去掉了，它们是常数项。

先看约束条件。$C \geq \alpha^{(i)} \geq 0, i=1,2,\cdots,N$是一个box constraint，将$\alpha$限制在了一个盒子中，包括$\alpha^{(1)}$和$\alpha^{(2)}$，如下图。

等式约束$\alpha^{(1)}y^{(1)} +  \alpha^{(2)}y^{(2)}  = \zeta$将$\alpha^{(1)}$和$\alpha^{(2)}$只能在平行于盒子对角线的线段上移动

<img src='img/二变量优化问题图示.png'>

图中是以$\alpha^{(1)}$为横轴，$\alpha^{(2)}$为纵轴。

左图和右图，区别在于选出的变量$\alpha^{(1)}$和$\alpha^{(2)}$，对应的样本点的标签$y^{(1)}$和$y^{(2)}$是否一致来决定。

当两个标签的符号不同时，对应左图。假设$y^{(1)}=1$，$y^{(2)}=-1$，则等式约束变成$\alpha^{(1)} -  \alpha^{(2)}  = \zeta$，移项后$\alpha^{(2)} =  \alpha^{(1)}  - \zeta$，这是一条斜率为正45度，截距为$-\zeta$的直线。当$y^{(1)}=-1$，$y^{(2)}=1$时，只需要提出一个负号，可知斜率不变，截距变成$\zeta$。

当两个标签的符号相同时，对应右图。假设$y^{(1)}=1$，$y^{(2)}=1$，则等式约束变成$\alpha^{(1)} +  \alpha^{(2)}  = \zeta$，移项后$\alpha^{(2)} =  -\alpha^{(1)}  + \zeta$，这是一条斜率为负45度，截距为$\zeta$的直线。当$y^{(1)}=-1$，$y^{(2)}=-1$时，只需要提出一个负号，可知斜率不变，截距变成$-\zeta$。

可以看到实际上只需要求解一个变量，另外一个变量就可以根据约束条件得出下面以求解$\alpha^{(2)}$为例。假设原来的变量是$(\alpha^{(1)})^{\text{old}}$和$(\alpha^{(2)})^{\text{old}}$。首先判断$\alpha^{(2)}$。求解出的变量是$(\alpha^{(1)})^{\text{new}}$和$(\alpha^{(2)})^{\text{new}}$。其中要得到$(\alpha^{(2)})^{\text{new}}$，必须要使得经过解析方法求解出的变量满足约束条件，即$L \leq(\alpha^{(2)})^{\text{new}} \leq H$，其中$L$和$H$分别是$(\alpha^{(2)})^{\text{new}}$的上下限。这里再定义，没有经过约束条件剪辑的$\alpha^{(2)}$为$(\alpha^{(2)})^{\text{new,unc}}$。下面讨论$L$和$H$的取值。

当$y^{(1)}\neq y^{(2)}$时，则$L=\max(0, (\alpha^{(2)})^{\text{old}}-(\alpha^{(1)})^{\text{old}})$, $H=\min(C, C +(\alpha^{(2)})^{\text{old}}-(\alpha^{(1)})^{\text{old}})$。

当$y^{(1)}= y^{(2)}$时，则$L=\max(0, (\alpha^{(2)})^{\text{old}}+(\alpha^{(1)})^{\text{old}}-C)$, $H=\min(C, (\alpha^{(2)})^{\text{old}}+(\alpha^{(1)})^{\text{old}})$。

**下面推导求解$(\alpha^{(2)})^{\text{new,unc}}$的解析过程**

定义
$$
g(\textbf{x})=\sum_{i=1}^N\alpha^{(i)}y^{(i)}K(\textbf{x}^{(i)}, \textbf{x}) + b
$$
这是分离超平面用$\alpha$表达的函数。
定义误差
$$
E^{(j)} = g(\textbf{x}^{(j)}) - y^{(j)} = (\sum_{i=1}^N\alpha^{(i)}y^{(i)}K(\textbf{x}^{(i)}, \textbf{x}^{(j)}) + b)-y^{(j)}, \quad j=1,2
$$
这是分离函数对$\textbf{x}^{(j)}$的预测值与真实值$y^{(j)}$之间的误差。

定义
$$
\begin{aligned}
\nu^{(j)} &= \sum_{i=3}^N{\alpha^{(i)}y^{(i)}K(\textbf{x}^{(i)}, \textbf{x}^{(j)})} \\
&= g(\textbf{x}^{(j)}) - \sum_{i=1}^2{\alpha^{(i)}y^{(i)}K(\textbf{x}^{(i)}, \textbf{x}^{(j)})} - b
\quad j=1,2
\end{aligned}
$$

带入$W(\alpha^{(1)}, \alpha^{(2)})$，把其中带有从i=3到N的求和项替换，得到
$$
\begin{aligned}
W(\alpha^{(1)}, \alpha^{(2)}) =  &\frac{1}{2}K(\textbf{x}^{(1)}, \textbf{x}^{(1)}) (\alpha^{(1)})^2 + \frac{1}{2}K(\textbf{x}^{(2)}, \textbf{x}^{(2)}) (\alpha^{(2)})^2  + y^{(1)}y^{(2)}K(\textbf{x}^{(1)}, \textbf{x}^{(2)}) \alpha^{(1)}\alpha^{(2)} \\
&- \alpha^{(1)} - \alpha^{(2)} + y^{(1)}\alpha^{(1)}\nu^{(1)} + y^{(2)}\alpha^{(2)}\nu^{(2)}
\end{aligned}
$$

因为$\alpha^{(1)}y^{(1)} +  \alpha^{(2)}y^{(2)} = \zeta$，两边同时乘以$y^{(1)}$，得到$\alpha^{(1)}(y^{(1)})^2 +  \alpha^{(2)}y^{(2)}y^{(1)} = \zeta y^{(1)}$。其中$(y^{(1)})^2=1$。整理后得到
$$
\alpha^{(1)} = (\zeta - \alpha^{(2)}y^{(2)})y^{(1)}
$$


将$\alpha^{(1)}$用$\alpha^{(2)}$来表示，代回$W(\alpha^{(1)}, \alpha^{(2)})$中，再根据$(y^{(2)})^2=1$，整理得
$$
\begin{aligned}
&\frac{1}{2}K(\textbf{x}^{(1)}, \textbf{x}^{(1)})(\zeta - \alpha^{(2)}y^{(2)})^2 + \frac{1}{2}K(\textbf{x}^{(2)}, \textbf{x}^{(2)}) (\alpha^{(2)})^2 + y^{(2)}K(\textbf{x}^{(1)}, \textbf{x}^{(2)}) \alpha^{(2)}(\zeta - \alpha^{(2)}y^{(2)}) \\
& -  (\zeta - \alpha^{(2)}y^{(2)})y^{(1)} - \alpha^{(2)} + (\zeta - \alpha^{(2)}y^{(2)})\nu^{(1)} + y^{(2)}\alpha^{(2)}\nu^{(2)}
\end{aligned}
$$

核函数看成常数，可以看到只有一个变量$\alpha^{(2)}$，可以标记目标函数用$W(\alpha^{(2)})$表示。

对$\alpha^{(2)}$求导，得到
$$
\begin{aligned}
\frac{\partial W}{\partial \alpha^{(2)}}= &-K(\textbf{x}^{(1)}, \textbf{x}^{(1)})\zeta y^{(2)} + K(\textbf{x}^{(1)}, \textbf{x}^{(1)})\alpha^{(2)} + K(\textbf{x}^{(2)}, \textbf{x}^{(2)}) \alpha^{(2)} + y^{(2)}K(\textbf{x}^{(1)}, \textbf{x}^{(2)}) \zeta \\
&- 2K(\textbf{x}^{(1)}, \textbf{x}^{(2)})\alpha^{(2)} + y^{(1)}y^{(2)} - 1 - \nu^{(1)}y^{(2)} + y^{(2)}\nu^{(2)}
\end{aligned}
$$

令$\frac{\partial W}{\partial \alpha^{(2)}}=0$，整理两边，把$\alpha^{(2)}$整理出来，因为这里的自变量是$\alpha^{(2)}$，整理得
$$
\begin{aligned}
(K(\textbf{x}^{(1)}, \textbf{x}^{(1)}) + K(\textbf{x}^{(2)}, \textbf{x}^{(2)}) - 2K(\textbf{x}^{(1)}, \textbf{x}^{(2)})) \alpha^{(2)} &= K(\textbf{x}^{(1)}, \textbf{x}^{(1)})\zeta y^{(2)} - y^{(2)}K(\textbf{x}^{(1)}, \textbf{x}^{(2)}) \zeta -  y^{(1)}y^{(2)} + 1 +  \nu^{(1)}y^{(2)} - y^{(2)}\nu^{(2)} \\
&= y^{(2)}(K(\textbf{x}^{(1)}, \textbf{x}^{(1)})\zeta  - K(\textbf{x}^{(1)}, \textbf{x}^{(2)}) \zeta -  y^{(1)} + y^{(2)} +  \nu^{(1)} - \nu^{(2)}) \\
\end{aligned}
$$

将$\nu^{(j)} = g(\textbf{x}^{(j)}) - \sum_{i=1}^2{\alpha^{(i)}y^{(i)}K(\textbf{x}^{(i)}, \textbf{x}^{(j)})} - b$带入整理得到
$$
\begin{aligned}
(K(\textbf{x}^{(1)}, \textbf{x}^{(1)}) + K(\textbf{x}^{(2)}, \textbf{x}^{(2)}) - 2K(\textbf{x}^{(1)}, \textbf{x}^{(2)})) \alpha^{(2)} &= K(\textbf{x}^{(1)}, \textbf{x}^{(1)})\zeta y^{(2)} - y^{(2)}K(\textbf{x}^{(1)}, \textbf{x}^{(2)}) \zeta -  y^{(1)}y^{(2)} + 1 +  \nu^{(1)}y^{(2)} - y^{(2)}\nu^{(2)} \\
&= y^{(2)}[K(\textbf{x}^{(1)}, \textbf{x}^{(1)})\zeta  - K(\textbf{x}^{(1)}, \textbf{x}^{(2)}) \zeta -  y^{(1)} + y^{(2)} +  \\
&(g(\textbf{x}^{(1)}) - \sum_{i=1}^2{\alpha^{(i)}y^{(i)}K(\textbf{x}^{(i)}, \textbf{x}^{(1)})} - b) - (g(\textbf{x}^{(2)}) - \sum_{i=1}^2{\alpha^{(i)}y^{(i)}K(\textbf{x}^{(i)}, \textbf{x}^{(2)})} - b)]
\end{aligned}
$$

利用$\alpha^{(1)}y^{(1)} +  \alpha^{(2)}y^{(2)} = \zeta$替换等式右边的$\zeta$。注意，这里因为我们要求$\alpha^{(2)}$，是解析求法，得到的是$(\alpha^{(2)})^{\text{new,unc}}$，所以等式右边替换了$\zeta$是用old的$\alpha$，即$(\alpha^{(1)})^{\text{old}}y^{(1)} +  (\alpha^{(2)})^{\text{old}}y^{(2)} = \zeta$，无论是old还是new的$\alpha^{(1)}$或$\alpha^{(2)}$都会满足这个约束条件。带入后整理得到

$$
\begin{aligned}
(K(\textbf{x}^{(1)}, \textbf{x}^{(1)}) + K(\textbf{x}^{(2)}, \textbf{x}^{(2)}) - 2K(\textbf{x}^{(1)}, \textbf{x}^{(2)})) (\alpha^{(2)})^{\text{new,unc}} =& 
y^{(2)}[K(\textbf{x}^{(1)}, \textbf{x}^{(1)})((\alpha^{(1)})^{\text{old}}y^{(1)} +  (\alpha^{(2)})^{\text{old}}y^{(2)})  - K(\textbf{x}^{(1)}, \textbf{x}^{(2)}) ((\alpha^{(1)})^{\text{old}}y^{(1)} +  (\alpha^{(2)})^{\text{old}}y^{(2)}) \\ 
&- y^{(1)} + y^{(2)} + (g(\textbf{x}^{(1)}) - \sum_{i=1}^2{\alpha^{(i)}y^{(i)}K(\textbf{x}^{(i)}, \textbf{x}^{(1)})} - b) - (g(\textbf{x}^{(2)}) - \sum_{i=1}^2{\alpha^{(i)}y^{(i)}K(\textbf{x}^{(i)}, \textbf{x}^{(2)})} - b)] \\
=& y^{(2)}[K(\textbf{x}^{(1)}, \textbf{x}^{(1)})(\alpha^{(1)})^{\text{old}}y^{(1)} + K(\textbf{x}^{(1)}, \textbf{x}^{(1)}) (\alpha^{(2)})^{\text{old}}y^{(2)} \\
&-K(\textbf{x}^{(1)}, \textbf{x}^{(2)})(\alpha^{(1)})^{\text{old}}y^{(1)} - K(\textbf{x}^{(1)}, \textbf{x}^{(2)}) (\alpha^{(2)})^{\text{old}}y^{(2)} \\
& + g(\textbf{x}^{(1)}) - y^{(1)} - (g(\textbf{x}^{(2)}) - y^{(2)}) \\
& - (K(\textbf{x}^{(1)}, \textbf{x}^{(1)})(\alpha^{(1)})^{\text{old}}y^{(1)} + K(\textbf{x}^{(2)}, \textbf{x}^{(1)})(\alpha^{(2)})^{\text{old}}y^{(2)}) \\ 
& + (K(\textbf{x}^{(1)}, \textbf{x}^{(2)})(\alpha^{(1)})^{\text{old}}y^{(1)} + K(\textbf{x}^{(2)}, \textbf{x}^{(2)})(\alpha^{(2)})^{\text{old}}y^{(2)})
] \\
=& y^{(2)}[(K(\textbf{x}^{(1)}, \textbf{x}^{(1)}) + K(\textbf{x}^{(2)}, \textbf{x}^{(2)}) - 2K(\textbf{x}^{(1)}, \textbf{x}^{(2)}))(\alpha^{(2)})^{\text{old}}y^{(2)} + E^{(1)}-E^{(2)}] \\
=&(K(\textbf{x}^{(1)}, \textbf{x}^{(1)}) + K(\textbf{x}^{(2)}, \textbf{x}^{(2)}) - 2K(\textbf{x}^{(1)}, \textbf{x}^{(2)}))(\alpha^{(2)})^{\text{old}} + y^{(2)}(E^{(1)}-E^{(2)})
\end{aligned}
$$

定义$\eta=K(\textbf{x}^{(1)}, \textbf{x}^{(1)}) + K(\textbf{x}^{(2)}, \textbf{x}^{(2)}) - 2K(\textbf{x}^{(1)}, \textbf{x}^{(2)})$，则上面的方程可以变为
$$
\begin{aligned}
\eta (\alpha^{(2)})^{\text{new,unc}} = \eta (\alpha^{(2)})^{\text{old}} + y^{(2)}(E^{(1)}-E^{(2)})
\end{aligned}
$$
所以
$$
\begin{aligned}
(\alpha^{(2)})^{\text{new,unc}} = (\alpha^{(2)})^{\text{old}} + \frac{y^{(2)}(E^{(1)}-E^{(2)})}{\eta}
\end{aligned}
$$

这部分推导看起来很是繁琐，但是无非就是一些等式的带入以及整理，式子中的每一项都是标量的运算，因此仔细推导后自己也可以得出结论。

计算完了$(\alpha^{(2)})^{\text{new,unc}}$，则要把它带入$L$和$H$的范围内进行剪辑。则

$$
\begin{equation}
(\alpha^{(2)})^{\text{new}}=
 \begin{cases}
 H, &(\alpha^{(2)})^{\text{new,unc}} > H \\
 (\alpha^{(2)})^{\text{new,unc}}, &L \leq (\alpha^{(2)})^{\text{new,unc}} \leq H \\
 L, & (\alpha^{(2)})^{\text{new,unc}} < L
   \end{cases}
\end{equation}
$$

根据
$$
(\alpha^{(1)})^{\text{old}}y^{(1)} +  (\alpha^{(2)})^{\text{old}}y^{(2)} = (\alpha^{(1)})^{\text{new}}y^{(1)} +  (\alpha^{(2)})^{\text{new}}y^{(2)}
$$
两边同时乘以$y^{(1)}$，再整理后得到
$$
(\alpha^{(1)})^{\text{new}} = y^{(1)}y^{(2)}((\alpha^{(2)})^{\text{old}}-(\alpha^{(2)})^{\text{new}}) + (\alpha^{(1)})^{\text{old}}
$$

以上就是求解$(\alpha^{(1)})^{\text{new}}$和$(\alpha^{(2)})^{\text{new}}$的解析求解方法。

### 4.3 两个变量的选择方法
**第一个变量的选择**
选择第一个变量$\alpha^{(1)}$的过程称为外层循环。首先检查在间隔边界上的支撑向量点，即$0 < \alpha^{(i)} < C$的样本点（不能取等于C的原因是这种情况数据集可能在间隔边界上，也有可能在间隔与分离超平面之间，甚至在错误的一侧）。检测它们是否满足KKT条件。如果都满足，则在整个数据集上着，看是否满足KKT条件。选择违反KKT条件最严重的样本点。

（《统计学习方法》中说选择$0 < \alpha^{(i)} < C$的样本点是因为根据KKT条件可知$y^{(i)}g(\textbf{x}^{i})=1$，其中$g(\textbf{x}^{(i)})=\sum_{j=1}^N{\alpha^{(j)}y^{(j)}K(\textbf{x}^{(i)},\textbf{x}^{(j)})} + b$。但是这里有一个疑问是KKT条件是一组解是问题最优解的充要条件，然而这里的$\alpha^{(i)}$并不是这个数据集的最优解，即目前初始化的这组$\alpha$对应的超平面只是一个普通的超平面，并没有做到间隔最大，没有必要满足KKT条件，因此这里选择$0 < \alpha^{(i)} < C$个人有点疑问。）

**第二个变量的选择**
选择第二个变量$\alpha^{(1)}$的过程称为内层循环。标准是希望能使$\alpha^{(2)}$的变化足够大。回顾计算$(\alpha^{(2)})^{\text{new,unc}}$的公式。
$$
(\alpha^{(2)})^{\text{new,unc}} = (\alpha^{(2)})^{\text{old}} + \frac{y^{(2)}(E^{(1)}-E^{(2)})}{\eta}
$$

从$(\alpha^{(2)})^{\text{old}}$和$(\alpha^{(2)})^{\text{new,unc}}$之间的变化是$\frac{y^{(2)}(E^{(1)}-E^{(2)})}{\eta}$。所以要使得$\alpha^{(2)}$的变化足够大，可以让$E^{(1)}-E^{(2)}$足够大。$E^{(1)}$由外层循环选择的$\alpha^{(1)}$确定。那$\alpha^{(2)}$选择了，就随即确定了$E^{(2)}$。因此在选择$\alpha^{(2)}$时，要根据$E^{(1)}$的符号作为依据。如果$E^{(1)}$为正，则选择最负或最小的$E^{(i)}$作为$E^{(2)}$。反之如果$E^{(1)}$为负，则选择最正或最大的$E^{(i)}$作为$E^{(2)}$。

这是一种理想的选取方法。具体的效果还是要看最小化目标函数这个任务是否正在执行，因为有特殊情况，即使采用上述的选取方法，目标函数得不到足够的下降。此时，选取$(\alpha^{(2)})$的范围扩大。先在支撑向量点的范围内依次寻找，找到一个可以使得目标函数有足够下降的那个变量作为$(\alpha^{(2)})$。如果找不到，则再扩大范围，在整个数据集范围所对应的$(\alpha^{(i)})$。如果还不能使目标函数有足够的下降，则表明此时外层循环选择的$(\alpha^{(1)})$不好，退回去重新选择$(\alpha^{(1)})$。

**更新阈值b和误差值$E^{(i)}$**
每次更新完两个变量后，要检查$(\alpha^{1})^{(\text{new})}$和$(\alpha^{2})^{(\text{new})}$是否还满足$0 < \alpha^{(i)} < C$。根据KKT条件可知，$y^{(i)}g(\textbf{x}^{i})=1$，两边同时乘以$y^{(i)}$得到$g(\textbf{x}^{i})=y^{(i)}$，所以
$$g(\textbf{x}^{1})=\sum_{j=1}^N{\alpha^{(j)}y^{(j)}K(\textbf{x}^{(1)},\textbf{x}^{(j)})} + b=y^{(1)}$$
但是其中$\sum_{j=1}^N$中的第1和第2项要用新的$\alpha$来代替，因此会破坏原来的KKT等式约束条件，因此必须对b做出调整，$b^{\text{new}}$整理得到
$$
(b^{(1)})^{\text{new}} = y^{(1)} - \sum_{j=3}^N{\alpha^{(j)}y^{(j)}K(\textbf{x}^{(j)}, \textbf{x}^{(1)})} - (\alpha^{(1)})^{\text{new}}y^{(1)}K(\textbf{x}^{(1)}, \textbf{x}^{(1)}) - (\alpha^{(2)})^{\text{new}}y^{(2)}K(\textbf{x}^{(2)}, \textbf{x}^{(1)})
$$

又因为$E^{(i)} = g(\textbf{x}^{(i)}) - y^{(i)}$，老的误差项
$$
(E^{(1)})^{\text{old}} = \sum_{i=3}^N{\alpha^{(i)}y^{(i)}K(\textbf{x}^{(i)}, \textbf{x}^{(1)})} + (\alpha^{(1)})^{\text{old}}y^{(1)}K(\textbf{x}^{(1)}, \textbf{x}^{(1)}) + (\alpha^{(2)})^{\text{old}}y^{(2)}K(\textbf{x}^{(2)}, \textbf{x}^{(1)}) + b^{\text{old}} - y^{(1)}
$$
移项得
$$
y^{(1)} -  \sum_{i=3}^N{\alpha^{(i)}y^{(i)}K(\textbf{x}^{(i)}, \textbf{x}^{(1)})} =  -(E^{(1)})^{\text{old}} + (\alpha^{(1)})^{\text{old}}y^{(1)}K(\textbf{x}^{(1)}, \textbf{x}^{(1)}) + (\alpha^{(2)})^{\text{old}}y^{(2)}K(\textbf{x}^{(2)}, \textbf{x}^{(1)}) + b^{\text{old}}
$$

把这两项抽出来是因为要把它们带入上面$(b^{(1)})^{\text{new}}$的表达式中去替换前两项，这样的好处是可以引入$b^{\text{new}}$和$b^{\text{old}}$的关系。整理得到

$$
\begin{aligned}
(b^{(1)})^{\text{new}} &= -(E^{(1)})^{\text{old}} + (\alpha^{(1)})^{\text{old}}y^{(1)}K(\textbf{x}^{(1)}, \textbf{x}^{(1)}) + (\alpha^{(2)})^{\text{old}}y^{(2)}K(\textbf{x}^{(2)}, \textbf{x}^{(1)}) + b^{\text{old}} -  (\alpha^{(1)})^{\text{new}}y^{(1)}K(\textbf{x}^{(1)}, \textbf{x}^{(1)}) - (\alpha^{(2)})^{\text{new}}y^{(2)}K(\textbf{x}^{(2)}, \textbf{x}^{(1)}) \\
&= -(E^{(1)})^{\text{old}} - y^{(1)}K(\textbf{x}^{(1)}, \textbf{x}^{(1)})[(\alpha^{(1)})^{\text{new}} - (\alpha^{(1)})^{\text{old}}] - y^{(2)}K(\textbf{x}^{(2)}, \textbf{x}^{(1)})[(\alpha^{(2)})^{\text{new}} - (\alpha^{(2)})^{\text{old}}] +  b^{\text{old}}
\end{aligned}
$$

同理，若$0 < (\alpha^{(2)})^{\text{new}} < C$，则
$$
\begin{aligned}
(b^{(2)})^{\text{new}} &= -(E^{(2)})^{\text{old}} - y^{(1)}K(\textbf{x}^{(1)}, \textbf{x}^{(2)})[(\alpha^{(1)})^{\text{new}} - (\alpha^{(1)})^{\text{old}}] - y^{(2)}K(\textbf{x}^{(2)}, \textbf{x}^{(2)})[(\alpha^{(2)})^{\text{new}} - (\alpha^{(2)})^{\text{old}}] +  b^{\text{old}}
\end{aligned}
$$

如果$(\alpha^{1})^{(\text{new})}$和$(\alpha^{2})^{(\text{new})}$同时满足$0 < \alpha^{(i)} < C$，则$(b^{(1)})^{\text{new}}=(b^{(2)})^{\text{new}}=b^{\text{new}}$。否则$b^{\text{new}}$取$(b^{(1)})^{\text{new}}$和$(b^{(2)})^{\text{new}}$的均值。

更新$(E^{(i)})^{\text{new}}$，根据误差项的定义，